In [24]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.offline import iplot, init_notebook_mode
import plotly.io as pio
import glob, os, sys
import logging
from sys import argv
import re
import csv
from scipy.signal import find_peaks
from pathlib import Path
pio.orca.config.executable = str(Path('C:/Users/Weugene/AppData/Local/Programs/orca/orca.exe'))
np.set_printoptions(threshold=sys.maxsize)
import scipy.spatial.distance as ds


rpath = 'C:/Users/Weugene/AppData/Local/Packages/CanonicalGroupLimited.UbuntuonWindows_79rhkp1fndgsc/LocalState/rootfs/home/e.sharaborin/basilisk/work/mass_conservation/COMPARISON/'
files_csv = ['m0.08',  'm0.125',  'm0.2']
names = ['r=0.08', 'r=0.125', 'r=0.2' ]
ynames = ['V<sub>g</sub>', 'U<sub>mean</sub>', 'U<sub>max</sub>']
fnames = ['Vg', 'Umean', 'Umax']
lines = ['solid', 'dot', 'dash', 'longdash']
print ("path:", rpath)

#WINDOWS 10
path_ex = rpath
#MAC OS
# path_ex = '/Users/weugene/basilisk/work/tube/'
for i,f in enumerate(files_csv):
    files_csv[i] = rpath + files_csv[i]
### STYLE
yaxis = dict(
              tickfont = dict(
              family = 'Times New Roman',
              size = 20,
              color = 'black'
              ),
              titlefont = dict(
                  family = 'Times New Roman',
                  size = 25,
                  color = 'black'
              ),
          )
xaxis = dict(
              tickfont = dict(
                  family = 'Times New Roman',
                  size = 20,
                  color = 'black'
              ),
              titlefont = dict(
                  family = 'Times New Roman',
                  size = 25,
                  color = 'black'
              )
          )
axis_style = dict(showline=True, gridwidth=1, gridcolor='lightgrey', linewidth=2, linecolor='black', mirror=True, ticks='outside',
                 tickfont = dict(family = 'Times New Roman', size = 18, color = 'black'))
bg_style = {'plot_bgcolor': 'rgba(255, 255, 255, 1)', 'paper_bgcolor': 'rgba(255, 255, 255, 1)',}
col =   ['blue',           'red',        'hsv(120,100,100)',       'black' ]

### END of STYLE


headers1=['level', 'i', 't', 'dt','avggas','statuavg', 'staturms', ' statumax', 'NAN' ]

df_exp = []

for i, file in enumerate(files_csv):
    print('FILE name: ',file)
    df_exp.append(pd.read_csv(Path(file), header=None, sep=' '))
    df_exp[i].columns = headers1
for j,y in enumerate(['avggas', 'statuavg', ' statumax']):
    fig = go.Figure()
    for i, file in enumerate(files_csv):
        fig.add_trace(go.Scatter(x=df_exp[i]['t'], y=df_exp[i][y], name=names[i],
                             mode='lines+markers',
                             line=dict(color=col[i], width=2, dash = lines[i]),
                             textfont=dict(
                                family="sans serif",
                                size=18,
                                color="LightSeaGreen")
                             ))

    fig.update_layout(
        width = 1000,
        height = 500,
        xaxis_title='t',
        yaxis_title=ynames[j],
        yaxis = yaxis,
        xaxis = xaxis
    )
    fig.update_layout(bg_style)
    fig.update_xaxes(axis_style)
    fig.update_yaxes(axis_style)
    # fig.update_layout( yaxis_type = "log" )
    maxy = max(df_exp[:][y])
    fig.update_yaxes(range=[0,maxy])
    fig.update_xaxes(range=[0,0.4])
    fig.update_layout(showlegend=True)
    fig.update_layout(legend = dict(font = dict(family = "Times New Roman", size = 25, color = "black")),
                      legend_title = dict(font = dict(family = "Times New Roman", size = 25, color = "blue")))

    fig.update_layout(legend=dict(
        yanchor="bottom",
        y=0.1,
        xanchor="right",
        x=1,
        bordercolor="Black",
        borderwidth=2
    ))
    fig.update_layout(
        autosize=False,
        width=850,
        height=600,
        margin=dict(
            l=1,
            r=1,
            b=1,
            t=1,
            pad=4
        ),
    #     paper_bgcolor="LightSteelBlue",
    )

    if True:
        fn = rpath + fnames[j] + '.pdf'
        iplot(fig)
        pio.write_image(fig, str(Path(fn)))
        print("Successfully generated:", fn)


path: C:/Users/Weugene/AppData/Local/Packages/CanonicalGroupLimited.UbuntuonWindows_79rhkp1fndgsc/LocalState/rootfs/home/e.sharaborin/basilisk/work/mass_conservation/COMPARISON/
FILE name:  C:/Users/Weugene/AppData/Local/Packages/CanonicalGroupLimited.UbuntuonWindows_79rhkp1fndgsc/LocalState/rootfs/home/e.sharaborin/basilisk/work/mass_conservation/COMPARISON/m0.08
FILE name:  C:/Users/Weugene/AppData/Local/Packages/CanonicalGroupLimited.UbuntuonWindows_79rhkp1fndgsc/LocalState/rootfs/home/e.sharaborin/basilisk/work/mass_conservation/COMPARISON/m0.125
FILE name:  C:/Users/Weugene/AppData/Local/Packages/CanonicalGroupLimited.UbuntuonWindows_79rhkp1fndgsc/LocalState/rootfs/home/e.sharaborin/basilisk/work/mass_conservation/COMPARISON/m0.2


TypeError: list indices must be integers or slices, not str

In [30]:
import numpy as np
import pandas as pd
from mpl_toolkits.mplot3d import axes3d
import plotly.express as px
import plotly.graph_objects as go
from matplotlib import cm
from scipy.interpolate import griddata
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from scipy import interpolate
import matplotlib.ticker as ticker
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

#Plot numbers without .0. i.e. 1.000 => just 1
class nf(float):
    def __repr__(self):
        s = f'{self:.1f}'
        return f'{self:.0f}' if s[-1] == '0' else s

    

df_dict = pd.read_excel('https://drive.google.com/u/0/uc?id=1X7McO1BVxzI7ntRDWX3ycpKbFmqVUItg&export=download', header=[0, 1], index_col=0, sheet_name='Sheet2')
df=df_dict.rename_axis((None,None), axis=1)
df.columns.names = ['level', 'eta']
level_l=list(df.columns.get_level_values('level'))
eta_l=list(df.columns.get_level_values('eta'))
level=list(df.columns.get_level_values('level').unique())
eta=list(df.columns.get_level_values('eta').unique())
phi = np.array([.05, .1, .2, .3, .4, .5, .6, .7, .75])

minErr=df.min().min()
maxErr=df.max().max()
levels_log = np.logspace(np.log10(minErr), np.log10(maxErr), 11) #Log of Error
levels = np.array([-3, -2.6, -2.2, -2, -1.5, -1.2, -1., -0.7, 0]) # log Error 
Nphi=len(phi)
Nlevel=len(level)
Neta=len(eta)
fi=list(df.index)
M = np.array(df).reshape(9, 4, 6) #PHI, LEVEL, ETA
minErr=M.min()
maxErr=M.max()



In [179]:
#Parameters
lev = 10
# etas = [1e-15, 1e-06, 0.0001, 0.001]
etas = [ 1e-03, 1e-04, 1e-05, 1e-06, 1e-15 ]
rpath = 'C:/Users/Weugene/AppData/Local/Packages/CanonicalGroupLimited.UbuntuonWindows_79rhkp1fndgsc/LocalState/rootfs/home/e.sharaborin/basilisk/work/cylinders_penalization/comparison_BP_Pt_EB/res10/'

# Plotly for Pandas
pd.options.plotting.backend = "plotly"
### STYLE
yaxis = dict(
              tickfont = dict(
              family = 'Times New Roman',
              size = 20,
              color = 'black'
              ),
              titlefont = dict(
                  family = 'Times New Roman',
                  size = 25,
                  color = 'black'
              ),
          )
xaxis = dict(
              tickfont = dict(
                  family = 'Times New Roman',
                  size = 20,
                  color = 'black'
              ),
              titlefont = dict(
                  family = 'Times New Roman',
                  size = 25,
                  color = 'black'
              )
          )
axis_style = dict(showline=True, gridwidth=1, gridcolor='lightgrey', linewidth=1, linecolor='black', mirror=True, ticks='outside',
                 tickfont = dict(family = 'Times New Roman', size = 18, color = 'black'))
bg_style = {'plot_bgcolor': 'rgba(255, 255, 255, 1)', 'paper_bgcolor': 'rgba(255, 255, 255, 1)',}
col =   ['blue',           'red',        'hsv(120,100,100)',       'black', 'magenta', 'goldenrod' ]
lines = ['solid', 'dot', 'dash', 'longdash', "dashdot", "longdashdot"]
mst = ['circle', 'triangle-up', 'triangle-down','square', 'diamond', 'cross',  'x-thin', 'cross-thin' ]
mst = ['circle-open', 'triangle-up-open', 'triangle-down-open', 'square-open', 'diamond-open', 'cross-open', 'x-thin-open', 'cross-thin-open' ]
### END of STYLE
# m = []
# for i,et in enumerate(eta):
#     print(int(np.sqrt(et)*2**lev))
#     m.append(max(5,int(np.sqrt(et)*2**lev)))
    
    
fig = go.Figure()
for j,lev in enumerate([8,9,10,11]):
    for i,et in enumerate(etas):
        m = min(int(np.sqrt(et)*2**lev), 5)
        if m == 5:
            nm = '≥ ' + str(m)
        else:
            nm = '= ' + str(m)
#         print(m)
        pw=np.log10(et)
#         fig.add_trace(go.Scatter(x=phi, y=df[lev][et].values, name=u"\u03B7=10<sup>" + str(int(pw))+'</sup>, J<sub>max</sub>='+str(lev)+', m ' + nm,
        fig.add_trace(go.Scatter(x=phi, y=df[lev][et].values, name='J<sub>max</sub>='+str(lev)+u" \u03B7=10<sup>" + str(int(pw))+'</sup>, m' + nm,

                             mode='lines+markers',
                             marker = dict(
                                     size = 15,
                                     line = dict(
                                      color = 'rgb(0, 0, 250)',
                                      width = 1
                                      )
                                 ),
                             marker_symbol = mst[m],
                             line=dict(width=2, dash = lines[i], color = col[j]
                                      ),
                             textfont=dict(
                                family="sans serif",
                                size=18,
                                color="LightSeaGreen")
                             ))

    
fig.update_layout(
    width = 1000,
    height = 500,
    xaxis_title=u'\u03c6<sub>s</sub>',
    yaxis_title='E',
    yaxis = yaxis,
    xaxis = xaxis
)
fig.update_layout(bg_style)
fig.update_xaxes(axis_style)
fig.update_yaxes(axis_style)
fig.update_layout( yaxis_type = "log" )
# fig.update_yaxes(range=[-5,0])
#     fig.update_xaxes(range=[xmin-x0,xmax-x0])
fig.update_layout(showlegend=True)
fig.update_layout(legend = dict(font = dict(family = "Times New Roman", size = 20, color = "black")),
                  legend_title = dict(font = dict(family = "Times New Roman", size = 20, color = "black")))


# fig.update_layout(legend_title_text=u"          \u03B7")
fig.update_layout(
    autosize=False,
    width=850,
    height=600,
    margin=dict(
        l=1,
        r=1,
        b=1,
        t=1,
        pad=4
    ),
#     paper_bgcolor="LightSteelBlue"
)
fig.update_layout(legend=dict(
    font = dict(family = "Times New Roman", size = 18, color = "black"),
    yanchor="bottom",
    y=0,
    xanchor="left",
    x=1.009,
    bordercolor="Black",
    borderwidth=0
))
if True:
    fn = rpath + 'logError_vs_phi_for_diff_eta_l=8-11.pdf'
    iplot(fig)
    pio.write_image(fig, str(Path(fn)))
    print("Successfully generated:", fn)




Successfully generated: C:/Users/Weugene/AppData/Local/Packages/CanonicalGroupLimited.UbuntuonWindows_79rhkp1fndgsc/LocalState/rootfs/home/e.sharaborin/basilisk/work/cylinders_penalization/comparison_BP_Pt_EB/res10/logError_vs_phi_for_diff_eta_l=8-11.pdf


In [34]:
type(df[10].plot())

plotly.graph_objs._figure.Figure

In [166]:
        fig.add_trace(go.Scatter(x=phi, y=df[lev][et].values, name='J<sub>max</sub>='+str(lev)+u" \u03B7=10<sup>" + str(int(pw))+'</sup>, m' + nm


-6

In [160]:
round(3.24)

3

In [44]:
import plotly.graph_objects as go
from scipy.interpolate import griddata


feature_x = eta_l
feature_y = phi
# Creating 2-D grid of features  
[X, Y] = np.meshgrid(feature_x, feature_y) 
Z = df[10][:]

points = np.random.rand(100, 2)
values = func(points[:,0], points[:,1])
f = interpolate.interp2d(x=points[:,0], y=points[:,1], z=values, kind='cubic')

# grid_z0 = griddata(points, values, (X, Y), method='cubic')  


fig = go.Figure(data =
    go.Contour(
        z=values,
        x=points[:,0], # horizontal axis
        y=points[:,1] # vertical axis
    ))

# fig = go.Figure(data =
#     go.Contour(
#         z=Z,
#         x=X, # horizontal axis
#         y=Y # vertical axis
#     ))
fig.show()

C:\Users\Weugene\Anaconda3\lib\site-packages\scipy\interpolate\_fitpack_impl.py:976: RuntimeWarning:

No more knots can be added because the number of B-spline
coefficients already exceeds the number of data points m.
Probable causes: either s or m too small. (fp>s)
	kx,ky=3,3 nx,ny=14,15 m=100 fp=0.000000 s=0.000000

